In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("../lib")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


In [2]:
import pickle
import lmdb
import cv2
import csv

In [3]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

../lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.

In [4]:
label_map_coco = label_map_util.load_labelmap(os.path.join('../lib/object_detection/data', 'mscoco_label_map.pbtxt'))
label_map_oi = label_map_util.load_labelmap(os.path.join('../lib/object_detection/data', 'oid_bbox_trainable_label_map.pbtxt'))


In [5]:
i = label_map_coco.item[0]

In [6]:
i

name: "/m/01g317"
id: 1
display_name: "person"

In [7]:
idx2mid_coco = {}
for item in label_map_coco.item:
    idx2mid_coco[item.id] = item.name

idx2mid_oi = {}
for item in label_map_oi.item:
    idx2mid_oi[item.id] = item.name



In [8]:
mid2idx_coco = {}
for item in label_map_coco.item:
    mid2idx_coco[item.name] = item.id

mid2idx_oi = {}
for item in label_map_oi.item:
    mid2idx_oi[item.name] = item.id



In [9]:
mid2name_coco = {}
for item in label_map_coco.item:
    mid2name_coco[item.name] = item.display_name

mid2name_oi = {}
for item in label_map_oi.item:
    mid2name_oi[item.name] = item.display_name



In [61]:
with open('../temp/label_matching_coco_oi.pkl', 'wb') as fout:
    pickle.dump({
        'idx2mid_coco': idx2mid_coco,
        'idx2mid_oi': idx2mid_oi,
        'mid2idx_coco': mid2idx_coco,
        'mid2idx_oi': mid2idx_oi,
        'mid2name_coco': mid2name_coco,
        'mid2name_oi': mid2name_oi,
    }, fout)

In [10]:
coco_mids = set(list(mid2idx_coco.keys()))
oi_mids = set(list(mid2idx_oi.keys()))

In [11]:
common_mids = coco_mids & oi_mids

In [12]:
len(common_mids), len(coco_mids - common_mids), len(oi_mids - common_mids)

(74, 6, 471)

In [13]:
[mid2name_coco[item] for item in coco_mids - common_mids]

['toaster', 'parking meter', 'frisbee', 'airplane', 'hair drier', 'cup']

In [14]:
[(mid2name_coco[item], mid2name_oi[item]) for item in common_mids]

[('sandwich', 'Sandwich'),
 ('dining table', 'Table'),
 ('cat', 'Cat'),
 ('traffic light', 'Traffic light'),
 ('toilet', 'Toilet'),
 ('microwave', 'Microwave oven'),
 ('scissors', 'Scissors'),
 ('oven', 'Oven'),
 ('potted plant', 'Houseplant'),
 ('spoon', 'Spoon'),
 ('pizza', 'Pizza'),
 ('hot dog', 'Hot dog'),
 ('donut', 'Doughnut'),
 ('laptop', 'Laptop'),
 ('toothbrush', 'Toothbrush'),
 ('sheep', 'Sheep'),
 ('suitcase', 'Suitcase'),
 ('baseball bat', 'Baseball bat'),
 ('umbrella', 'Umbrella'),
 ('bottle', 'Bottle'),
 ('sink', 'Sink'),
 ('clock', 'Clock'),
 ('tennis racket', 'Racket'),
 ('tv', 'Television'),
 ('handbag', 'Handbag'),
 ('bench', 'Bench'),
 ('keyboard', 'Computer keyboard'),
 ('carrot', 'Carrot'),
 ('person', 'Person'),
 ('motorcycle', 'Motorcycle'),
 ('fire hydrant', 'Fire hydrant'),
 ('bed', 'Bed'),
 ('cell phone', 'Mobile phone'),
 ('train', 'Train'),
 ('backpack', 'Backpack'),
 ('orange', 'Orange'),
 ('elephant', 'Elephant'),
 ('bear', 'Bear'),
 ('cake', 'Cake'),
 ('v

In [15]:
imglabels = {}
annot = {}
label2imgs = {}

with open('../data/OpenImages/metadata-v4/bbox/test-annotations-bbox.csv', 'r') as fin:
    for row in csv.reader(fin):
        imgid = row[0]
        imglb = row[2]
        if imglb in common_mids:
            if annot.get(imgid) == None:
                annot[imgid] = []
            annot[imgid].append({
                'label': imglb,
                'bbox': tuple([float(item) for item in row[4:8]]),
            })
            if label2imgs.get(imglb) is None:
                label2imgs[imglb] = set()
            label2imgs[imglb].add(imgid)
            if imglabels.get(imgid) == None:
                imglabels[imgid] = set()
            imglabels[imgid].add(imglb)
            

In [16]:
len(imglabels)

49847

In [17]:
imgids_array = np.asarray(list(imglabels.keys()))
np.random.seed(1234)
np.random.shuffle(imgids_array)

In [18]:
label_cnt = {key: 0 for key in common_mids}
min_cnt = 100
img_subset = []
for key in imgids_array:
    for l in (set(imglabels[key]) & common_mids):
        flag = False
        if label_cnt[l] < min_cnt:
            flag = True
    if flag:
        img_subset.append(key)
        for l in (set(imglabels[key]) & common_mids):
            label_cnt[l] += 1
    

In [19]:
len(img_subset)

5372

In [20]:
with open('../temp/test_subset_coco_1.pkl', 'wb') as fout:
    pickle.dump(img_subset, fout)

In [22]:
with open('../temp/oi_coco_common_mids.pkl', 'wb') as fout:
    pickle.dump(common_mids, fout)

In [21]:
label_cnt

{'/m/0l515': 105,
 '/m/04bcr3': 175,
 '/m/01yrx': 108,
 '/m/015qff': 55,
 '/m/09g1w': 55,
 '/m/0fx9l': 43,
 '/m/01lsmm': 6,
 '/m/029bxz': 51,
 '/m/03fp41': 103,
 '/m/0cmx8': 100,
 '/m/0663v': 101,
 '/m/01b9xk': 32,
 '/m/0jy4k': 90,
 '/m/01c648': 100,
 '/m/012xff': 11,
 '/m/07bgp': 100,
 '/m/01s55n': 42,
 '/m/03g8mr': 23,
 '/m/0hnnb': 53,
 '/m/04dr76w': 100,
 '/m/0130jx': 97,
 '/m/01x3z': 43,
 '/m/0dv9c': 19,
 '/m/07c52': 100,
 '/m/080hkjn': 100,
 '/m/0cvnqh': 0,
 '/m/01m2v': 106,
 '/m/0fj52s': 75,
 '/m/01g317': 488,
 '/m/04_sv': 100,
 '/m/01pns0': 56,
 '/m/03ssj5': 103,
 '/m/050k8': 102,
 '/m/07jdr': 100,
 '/m/01940j': 51,
 '/m/0cyhj_': 101,
 '/m/0bwd_0j': 100,
 '/m/01dws': 60,
 '/m/0fszt': 101,
 '/m/02s195': 101,
 '/m/01mzpv': 118,
 '/m/09tvcd': 101,
 '/m/04kkgm': 100,
 '/m/04ctx': 103,
 '/m/020lf': 77,
 '/m/01bjv': 100,
 '/m/0qjjc': 19,
 '/m/040b_t': 73,
 '/m/03bk1': 47,
 '/m/0898b': 60,
 '/m/03k3r': 100,
 '/m/018xm': 103,
 '/m/0199g': 102,
 '/m/015p6': 104,
 '/m/07r04': 100,
 '/m/06

In [23]:
with open('../results/det_results_coco_oi.pkl', 'rb') as fin:
    det_results_oi = pickle.load(fin)

In [24]:
with open('../results/det_results_coco_1.pkl', 'rb') as fin:
    det_results_coco = pickle.load(fin)

In [25]:
test_img_ids = img_subset

In [68]:
iou_thresh = 0.5
score_matches = {}
for imgid in test_img_ids:
    for ii in range(len(annot[imgid])):
        gt_label = annot[imgid][ii]['label']
        gt_bbox = np.asarray(annot[imgid][ii]['bbox'])[[2,0,3,1]]
        
        match_oi = []        
        for iii in range(det_results_oi[imgid]['num_detections']):
            det_label = idx2mid_oi.get(det_results_oi[imgid]['detection_classes'][iii])
            det_bbox = det_results_oi[imgid]['detection_boxes'][iii]
            det_score = det_results_oi[imgid]['detection_scores'][iii]

            if det_label == gt_label:
                x_d_len = det_bbox[2] - det_bbox[0]
                y_d_len = det_bbox[3] - det_bbox[1]
                x_t_len = gt_bbox[2] - gt_bbox[0]
                y_t_len = gt_bbox[3] - gt_bbox[1]
                x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))
                y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))
                iou = (x_int_len*y_int_len) / (x_d_len*y_d_len + x_t_len*y_t_len - x_int_len*y_int_len)
                if iou >= iou_thresh:
                    match_oi.append(det_score)
                    
        match_coco = []        
        for iii in range(det_results_coco[imgid]['num_detections']):
            det_label = idx2mid_coco.get(det_results_coco[imgid]['detection_classes'][iii])
            det_bbox = det_results_coco[imgid]['detection_boxes'][iii]
            det_score = det_results_coco[imgid]['detection_scores'][iii]

            if det_label == gt_label:
                x_d_len = det_bbox[2] - det_bbox[0]
                y_d_len = det_bbox[3] - det_bbox[1]
                x_t_len = gt_bbox[2] - gt_bbox[0]
                y_t_len = gt_bbox[3] - gt_bbox[1]
                x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))
                y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))
                iou = (x_int_len*y_int_len) / (x_d_len*y_d_len + x_t_len*y_t_len - x_int_len*y_int_len)
                if iou >= iou_thresh:
                    match_coco.append(det_score)
    
        if len(match_oi) > 0 and len(match_coco) > 0:
            if score_matches.get(gt_label) is None:
                score_matches[gt_label] = []
            score_matches[gt_label].append((
                sum(match_oi) / len(match_oi),
                sum(match_coco) / len(match_coco),
            ))

In [69]:
norm_coef = {}
for label in common_mids:
    if score_matches.get(label) is None or len(score_matches[label]) < 10:
        norm_coef[label] = None
        continue
    data_oi, data_coco = zip(*score_matches[label])
    mean_oi = np.mean(data_oi)
    std_oi = np.std(data_oi)
    mean_coco = np.mean(data_coco)
    std_coco = np.std(data_coco)
    norm_coef[label] = {'mean_oi': mean_oi, 'std_oi': std_oi, 'mean_coco': mean_coco, 'std_coco': std_coco}

In [70]:
norm_coef

{'/m/0l515': {'mean_oi': 0.24453721904197204,
  'std_oi': 0.23170631246609405,
  'mean_coco': 0.9255695554335215,
  'std_coco': 0.1326410801547722},
 '/m/04bcr3': {'mean_oi': 0.30384650234322264,
  'std_oi': 0.23751835900969676,
  'mean_coco': 0.7303521578249178,
  'std_coco': 0.22304217566309362},
 '/m/01yrx': {'mean_oi': 0.6540393707381251,
  'std_oi': 0.2776119094254477,
  'mean_coco': 0.9780693682886306,
  'std_coco': 0.0699646847123486},
 '/m/015qff': {'mean_oi': 0.4885919092482873,
  'std_oi': 0.33662533870337447,
  'mean_coco': 0.958378264437551,
  'std_coco': 0.10061380038541408},
 '/m/09g1w': None,
 '/m/0fx9l': None,
 '/m/01lsmm': None,
 '/m/029bxz': None,
 '/m/03fp41': {'mean_oi': 0.2641781068360483,
  'std_oi': 0.26321389265460327,
  'mean_coco': 0.8271755962672398,
  'std_coco': 0.19474689316301422},
 '/m/0cmx8': None,
 '/m/0663v': {'mean_oi': 0.326466151867789,
  'std_oi': 0.23913285776868295,
  'mean_coco': 0.9766479519841907,
  'std_coco': 0.07401571683007092},
 '/m/01b9

In [71]:
default_norm_coef = {
    key: np.mean([val[key] for val in norm_coef.values() if val is not None])
    for key in ['mean_oi', 'std_oi', 'mean_coco', 'std_coco']
}

In [72]:
default_norm_coef

{'mean_oi': 0.4383073024401371,
 'std_oi': 0.288292198679016,
 'mean_coco': 0.9285995488837365,
 'std_coco': 0.12049593838021926}

In [75]:
for key in norm_coef:
    if norm_coef[key] is None:
        norm_coef[key] = dict(default_norm_coef)
        
norm_coef['default'] = dict(default_norm_coef)

In [76]:
with open('../temp/norm_coef_1.pkl', 'wb') as fout:
    pickle.dump(norm_coef, fout)